# CLI - Command-line interface

Let's take a look at the `kissim` command-line interface (CLI) to encode a set of structures (from the [KLIFS](https://klifs.net/) database) and perform an all-against-all comparison. The CLI follows the same logic as the quick start Python interface as described in "API - Quick start Python interface".
  
![kissim CLI](../_static/kissim_docs_cli.png)

In [1]:
from pathlib import Path

In [2]:
# Path to this notebook
HERE = Path(_dh[-1])  # noqa: F821

## Encode structures into fingerprints

In [3]:
%%bash
kissim encode -h
# flake8-noqa-cell

usage: kissim encode [-h] -i INPUT [INPUT ...] -o OUTPUT [-l LOCAL]
                     [-c NCORES]

optional arguments:
  -h, --help            show this help message and exit
  -i INPUT [INPUT ...], --input INPUT [INPUT ...]
                        List of structure KLIFS IDs or path to txt file
                        containing structure KLIFS IDs.
  -o OUTPUT, --output OUTPUT
                        Path to output json file containing fingerprint data.
  -l LOCAL, --local LOCAL
                        Path to KLIFS download folder. If set local KLIFS data
                        is used, else remote KLIFS data.
  -c NCORES, --ncores NCORES
                        Number of cores. If 1 fingerprint generation in
                        sequence, else in parallel.


### `kissim encode` command

In [4]:
%%bash
kissim encode -i 109 118 12347 1641 3833 9122 -o fingerprints.json -l ../../kissim/tests/data/KLIFS_download/ -c 2
# flake8-noqa-cell

INFO:opencadd.databases.klifs.api:Set up local session...
INFO:opencadd.databases.klifs.local:Load overview.csv...
INFO:opencadd.databases.klifs.local:Load KLIFS_export.csv...
INFO:opencadd.databases.klifs.local:Merge both csv files...
INFO:opencadd.databases.klifs.local:Add paths to coordinate folders to structures...
INFO:opencadd.databases.klifs.local:Add KLIFS IDs to structures (uses remote since not available locally!)...
INFO:opencadd.databases.klifs.api:Local session is ready!
INFO:kissim.encoding.fingerprint_generator:GENERATE FINGERPRINTS
INFO:kissim.encoding.fingerprint_generator:Number of input structures: 6
INFO:kissim.encoding.fingerprint_generator:Fingerprint generation started at: 2021-03-29 14:58:04.664193
INFO:kissim.utils:Number of cores used: 2.
INFO:kissim.encoding.fingerprint_generator:118: Generate fingerprint...
INFO:kissim.encoding.fingerprint_generator:109: Generate fingerprint...
INFO:kissim.encoding.fingerprint_generator:12347: Generate fingerprint...
INFO:ki

This command generate two files:

- Data: `fingerprints.json`
- Logs: `fingerprint.log`

### Inspect output: `FingerprintGenerator`

You can load the content of the `fingerprints.json` file as `FingerprintGenerator` object.

In [5]:
fingerprints_path = HERE / "fingerprints.json"
fingerprints_path

PosixPath('/home/dominique/Documents/GitHub/kissim/docs/tutorials/fingerprints.json')

In [6]:
from kissim.encoding import FingerprintGenerator

fingerprint_generator = FingerprintGenerator.from_json(fingerprints_path)
print(f"Number of fingerprints: {len(fingerprint_generator.data.keys())}")
fingerprint_generator

Number of fingerprints: 6


## Compare fingerprints

In [7]:
%%bash
kissim compare -h
# flake8-noqa-cell

usage: kissim compare [-h] -i INPUT -o OUTPUT
                      [-w WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS]
                      [-c NCORES]

optional arguments:
  -h, --help            show this help message and exit
  -i INPUT, --input INPUT
                        Path to json file containing fingerprint data.
  -o OUTPUT, --output OUTPUT
                        Path to output folder where distance json files will
                        be saved.
  -w WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS, --weights WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS WEIGHTS
                        Feature weights. Eeach feature must be set
                        individually, all weights must sum up to 1.0.
  -c NCORES, --ncores NCORES
                        Number of cor

### `kissim compare` command

In [8]:
%%bash
kissim compare -i fingerprints.json -o . -c 2
# flake8-noqa-cell

INFO:kissim.comparison.feature_distances_generator:GENERATE FEATURE DISTANCES
INFO:kissim.comparison.feature_distances_generator:Number of input input fingerprints: 6
INFO:kissim.comparison.feature_distances_generator:Feature distances generation started at: 2021-03-29 14:58:16.503597
INFO:kissim.utils:Number of cores used: 2.
INFO:kissim.comparison.feature_distances_generator:Number of ouput feature distances: 15
INFO:kissim.comparison.feature_distances_generator:Runtime: 0:00:00.061052
INFO:kissim.comparison.fingerprint_distance_generator:GENERATE FINGERPRINT DISTANCES
INFO:kissim.comparison.fingerprint_distance_generator:Number of input feature distances: 15
INFO:kissim.comparison.fingerprint_distance_generator:Fingerprint distance generation started at: 2021-03-29 14:58:16.565221
INFO:kissim.utils:Number of cores used: 2.
INFO:kissim.comparison.fingerprint_distance_generator:Feature weights: [0.06666667 0.06666667 0.06666667 0.06666667 0.06666667 0.06666667
 0.06666667 0.06666667 0

This command generate three files:

- Data (fingerprint distances): `fingerprint_distances.json`
- Data (feature distances): `feature_distances.json`
- Logs: `distances.log`

### Inspect output: `FingerprintDistanceGenerator`

You can load the content of the `fingerprint_distances.json` file as `FingerprintDistancesGenerator` object.

In [9]:
from kissim.comparison import FingerprintDistanceGenerator

fingerprint_distance_path = HERE / "fingerprint_distances.json"
fingerprint_distance_generator = FingerprintDistanceGenerator.from_json(fingerprint_distance_path)
print(f"Number of pairwise comparisons: {len(fingerprint_distance_generator.data)}")
fingerprint_distance_generator

Number of pairwise comparisons: 15


In [10]:
fingerprint_distance_generator.data

,structure1,structure2,kinase1,kinase2,distance,coverage
0,109,118,ABL2,ABL2,0.074214,0.992000
1,109,12347,ABL2,BRAF,0.259053,0.919333
2,109,1641,ABL2,CHK1,0.253045,0.990667
3,109,3833,ABL2,AAK1,0.277368,0.990667
4,109,9122,ABL2,ADCK3,0.358882,0.990667
5,118,12347,ABL2,BRAF,0.273133,0.918000
6,118,1641,ABL2,CHK1,0.246844,0.989333
7,118,3833,ABL2,AAK1,0.282949,0.990000
8,118,9122,ABL2,ADCK3,0.360833,0.989333
9,12347,1641,BRAF,CHK1,0.303330,0.918000


### Inspect output: `FeatureDistancesGenerator`

You can load the content of the `feature_distances.json` file as `FeatureDistancesGenerator` object.

In [11]:
from kissim.comparison import FeatureDistancesGenerator

feature_distances_path = HERE / "feature_distances.json"
feature_distances_generator = FeatureDistancesGenerator.from_json(feature_distances_path)
print(f"Number of pairwise comparisons: {len(feature_distances_generator.data)}")
feature_distances_generator

Number of pairwise comparisons: 15


In [12]:
feature_distances_generator.data

{(109,
  118): <kissim.comparison.feature_distances.FeatureDistances at 0x7f9970dcba60>,
 (109,
  12347): <kissim.comparison.feature_distances.FeatureDistances at 0x7f9910075040>,
 (109,
  1641): <kissim.comparison.feature_distances.FeatureDistances at 0x7f99100750a0>,
 (109,
  3833): <kissim.comparison.feature_distances.FeatureDistances at 0x7f9910075100>,
 (109,
  9122): <kissim.comparison.feature_distances.FeatureDistances at 0x7f9910075160>,
 (118,
  12347): <kissim.comparison.feature_distances.FeatureDistances at 0x7f99100751c0>,
 (118,
  1641): <kissim.comparison.feature_distances.FeatureDistances at 0x7f9910075220>,
 (118,
  3833): <kissim.comparison.feature_distances.FeatureDistances at 0x7f9910075280>,
 (118,
  9122): <kissim.comparison.feature_distances.FeatureDistances at 0x7f99100752e0>,
 (12347,
  1641): <kissim.comparison.feature_distances.FeatureDistances at 0x7f9910075340>,
 (12347,
  3833): <kissim.comparison.feature_distances.FeatureDistances at 0x7f99100753a0>,
 (123

Clean up output files.

In [13]:
[i.unlink() for i in HERE.glob("*.json")]
[i.unlink() for i in HERE.glob("*.log")]

[None, None]